#**Aula 1: Realizando a primeira previsão**

##**Conhecendo os dados**

> Você pode checar a documentação da biblioteca Pandas [aqui](https://pandas.pydata.org/).

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/alura-cursos/series_temporais_prophet/main/Dados/poluentes.csv')

In [ ]:
df

,Data,PM2.5,PM10,SO2,NO2,CO,O3,TEMP
0,2020-03-01,7.0,11.0,12.0,23.0,429.0,64.0,1.0
1,2020-03-02,31.0,42.0,37.0,67.0,825.0,30.0,1.0
2,2020-03-03,77.0,121.0,61.0,81.0,1621.0,19.0,6.0
3,2020-03-04,23.0,45.0,23.0,46.0,606.0,54.0,10.0
4,2020-03-05,149.0,184.0,94.0,133.0,2358.0,68.0,6.0
...,...,...,...,...,...,...,...,...
1455,2024-02-24,22.0,33.0,17.0,59.0,575.0,51.0,5.0
1456,2024-02-25,11.0,20.0,7.0,43.0,421.0,66.0,7.0
1457,2024-02-26,28.0,41.0,10.0,65.0,721.0,49.0,7.0
1458,2024-02-27,75.0,97.0,21.0,98.0,1427.0,37.0,8.0


In [ ]:
df['Data'] = pd.to_datetime(df['Data'])

In [ ]:
import plotly.express as px

fig = px.line(df, x='Data', y='O3')
fig.show()

In [ ]:
import calendar

# Criar um DataFrame com os dados de 2022
df_2022 = df[df['Data'].dt.year == 2022]

# Agrupar os dados por mês e calcule a média de O3
df_2022_mensal = df_2022.groupby(df_2022['Data'].dt.month)['O3'].mean().reset_index()

# Adicionar uma coluna com os nomes dos meses
df_2022_mensal['Mês'] = df_2022_mensal['Data'].apply(lambda x: calendar.month_abbr[x])

# Criar um gráfico de barras
fig = px.bar(df_2022_mensal, x='Mês', y='O3', title='Média de O3 ug/m3 por mês em 2022')

# Mostrar o gráfico
fig.show()

##**Realizando uma previsão**

> Você pode checar a documentação da biblioteca Prophet [aqui](https://facebook.github.io/prophet/).

In [ ]:
from prophet import Prophet

In [ ]:
df_prophet = pd.DataFrame()

df_prophet['ds'] = df['Data']
df_prophet['y'] = df['O3']

In [ ]:
df_prophet

,ds,y
0,2020-03-01,64.0
1,2020-03-02,30.0
2,2020-03-03,19.0
3,2020-03-04,54.0
4,2020-03-05,68.0
...,...,...
1455,2024-02-24,51.0
1456,2024-02-25,66.0
1457,2024-02-26,49.0
1458,2024-02-27,37.0


In [ ]:
# Definir um seed
import numpy as np
np.random.seed(4587)

# Instanciar o modelo Prophet
modelo = Prophet()

# Treinar o modelo
modelo.fit(df_prophet)

# Criar um dataframe para previsões futuras
futuro = modelo.make_future_dataframe(periods=365, freq='D')
previsao = modelo.predict(futuro)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpu064sqq2/ik8zk1rg.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpu064sqq2/m5q6atgm.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=36772', 'data', 'file=/tmp/tmpu064sqq2/ik8zk1rg.json', 'init=/tmp/tmpu064sqq2/m5q6atgm.json', 'output', 'file=/tmp/tmpu064sqq2/prophet_modeltfkaan6x/prophet_model-20240627160154.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
16:01:54 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
16:01:55 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [ ]:
from prophet.plot import plot_plotly
fig = plot_plotly(modelo, previsao)
fig.show()

In [ ]:
previsao

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2020-03-01,36.708750,-11.423571,49.879511,36.708750,36.708750,-17.570460,-17.570460,-17.570460,-2.094423,-2.094423,-2.094423,-15.476037,-15.476037,-15.476037,0.0,0.0,0.0,19.138290
1,2020-03-02,36.736524,-6.297502,54.194609,36.736524,36.736524,-13.465359,-13.465359,-13.465359,0.919980,0.919980,0.919980,-14.385338,-14.385338,-14.385338,0.0,0.0,0.0,23.271165
2,2020-03-03,36.764298,-7.189595,56.245281,36.764298,36.764298,-10.614908,-10.614908,-10.614908,2.599164,2.599164,2.599164,-13.214072,-13.214072,-13.214072,0.0,0.0,0.0,26.149390
3,2020-03-04,36.792072,-5.920984,55.542039,36.792072,36.792072,-11.796323,-11.796323,-11.796323,0.186921,0.186921,0.186921,-11.983244,-11.983244,-11.983244,0.0,0.0,0.0,24.995749
4,2020-03-05,36.819846,-3.379147,56.557718,36.819846,36.819846,-10.523998,-10.523998,-10.523998,0.190901,0.190901,0.190901,-10.714899,-10.714899,-10.714899,0.0,0.0,0.0,26.295848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1820,2025-02-23,62.126743,9.516483,69.112572,60.551676,63.523067,-21.557137,-21.557137,-21.557137,-2.094423,-2.094423,-2.094423,-19.462714,-19.462714,-19.462714,0.0,0.0,0.0,40.569606
1821,2025-02-24,62.124323,13.974234,73.743592,60.543791,63.529653,-18.292881,-18.292881,-18.292881,0.919980,0.919980,0.919980,-19.212860,-19.212860,-19.212860,0.0,0.0,0.0,43.831442
1822,2025-02-25,62.121903,16.612816,76.757365,60.535905,63.536142,-16.206558,-16.206558,-16.206558,2.599164,2.599164,2.599164,-18.805722,-18.805722,-18.805722,0.0,0.0,0.0,45.915345
1823,2025-02-26,62.119482,13.276886,73.417160,60.528020,63.538583,-18.058197,-18.058197,-18.058197,0.186921,0.186921,0.186921,-18.245118,-18.245118,-18.245118,0.0,0.0,0.0,44.061285


In [ ]:
previsao[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

,ds,yhat,yhat_lower,yhat_upper
0,2020-03-01,19.138290,-11.423571,49.879511
1,2020-03-02,23.271165,-6.297502,54.194609
2,2020-03-03,26.149390,-7.189595,56.245281
3,2020-03-04,24.995749,-5.920984,55.542039
4,2020-03-05,26.295848,-3.379147,56.557718
...,...,...,...,...
1820,2025-02-23,40.569606,9.516483,69.112572
1821,2025-02-24,43.831442,13.974234,73.743592
1822,2025-02-25,45.915345,16.612816,76.757365
1823,2025-02-26,44.061285,13.276886,73.417160


##**Analisando as componentes da série temporal**

In [ ]:
from prophet.plot import plot_components_plotly

plot_components_plotly(modelo, previsao)

A biblioteca Prophet usa um modelo aditivo que inclui componentes sazonais e de tendência. A equação utilizada pelo Prophet para realizar previsões pode ser representada da seguinte forma:

$y(t) = g(t) + s(t) + h(t) + \epsilon_t$

Onde:

- $ y(t)$ é o valor observado na série temporal no tempo \( t \).
- $ g(t) $ é a componente de tendência que modela mudanças de longo prazo.
- $ s(t) $ é a componente sazonal que modela padrões periódicos ou sazonais.
- $ h(t) $ é a componente de efeito de feriados
- $\epsilon_t$ é o erro aleatório na previsão.

Cada uma dessas componentes é modelada separadamente. A componente de tendência, $ g(t)$, é frequentemente modelada como uma função logística ou polinomial. A componente sazonal, $s(t)$, é modelada usando a análise de Fourier para capturar padrões periódicos. A componente de feriado, $h(t)$, é modelada como um efeito aditivo para cada feriado especificado.

Portanto, a equação completa usada pelo Prophet para realizar previsões é uma soma desses componentes mais o erro aleatório.

> Embora em nosso projeto não tenhamos detalhado a inclusão dos feriados no modelo, é importante ressaltar que essa possibilidade existe.